In [7]:
import pandas as pd
from google.colab import files
from nrclex import NRCLex
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.nn.functional import softmax
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
import ipywidgets as widgets
from IPython.display import display, clear_output

In [8]:
uploaded = files.upload()

Saving Tweets.csv to Tweets.csv


In [9]:
data = pd.read_csv("Tweets.csv")

In [10]:
nltk.download("punkt")
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [11]:
uploaded = files.upload()
data = pd.read_csv("Tweets.csv")
data = data.dropna(subset=["text"])

Saving Tweets.csv to Tweets (1).csv


In [12]:
def sentiment_vader(text):
  analyzer = SentimentIntensityAnalyzer()
  sentiment_scores = analyzer.polarity_scores(str(text))
  if sentiment_scores["compound"] >= 0.05:
    return "Positive"
  elif sentiment_scores["compound"] <= -0.05:
    return "Negative"
  else:
    return "Neutral"

In [13]:
def emotionanalysis(text):
  emotion_analyzer = NRCLex(str(text))
  top_emotions = emotion_analyzer.top_emotions
  return top_emotions

In [14]:
tokenizer = BertTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model_bert = BertForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

def sentiment_bert(text):
  inputs = tokenizer(str(text), return_tensors="pt")
  outputs = model_bert(**inputs)
  logits = outputs.logits
  predicted_label = torch.argmax(softmax(logits, dim=1)).item()
  sentiment_mapping = {0: "very negative", 1: "negative", 2: "neutral", 3: "positive", 4: "very positive"}
  return sentiment_mapping.get(predicted_label, "unknown")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [15]:
data["sentiment"] = data["text"].apply(sentiment_vader)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
nb_model = make_pipeline(CountVectorizer(), MultinomialNB())
nb_model.fit(train_data["text"].astype(str), train_data["sentiment"])

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('multinomialnb', MultinomialNB())])

In [16]:
text_input = widgets.Textarea(placeholder='Tweetinizi buraya yazın...', description='Tweet:', layout=widgets.Layout(width='600px', height='100px'))
analyze_button = widgets.Button(description="Analiz Et", button_style="success")
output = widgets.Output()

In [17]:
def on_analyze_clicked(b):
  with output:
    clear_output()
    tweet = text_input.value
    print("📌 Tweet:", tweet)
    print("🧠 NRC Duygu Analizi:", emotionanalysis(tweet))
    print("🗣️ VADER Sonuç:", sentiment_vader(tweet))
    print("🤖 BERT Sonuç:", sentiment_bert(tweet))
    print("📊 Naive Bayes Sonuç:", nb_model.predict([tweet])[0])

In [18]:
analyze_button.on_click(on_analyze_clicked)

In [19]:
display(text_input, analyze_button, output)

Textarea(value='', description='Tweet:', layout=Layout(height='100px', width='600px'), placeholder='Tweetinizi…

Button(button_style='success', description='Analiz Et', style=ButtonStyle())

Output()